# Logistic Regression to Classify Terrain by IMU and Odometry Data from TurtleBot3
### By Jacob Laframboise
Logistic regression works great when the data is randomly split into train and test (high 90 accuracy), but it struggles when the data is split into train/test based on which trial number is was collected in. This further supports the hypothesis that data from each run is more similar to itself than data from a given terrain. 



In [125]:
# imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

import plotly as ply
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
%matplotlib notebook

pd.set_option('display.max_columns', 120)
pd.set_option('display.max_rows', 80)

In [126]:
# load data
dataFolder = r"C:\Users\jaker\Documents\RoboData-Experiment1"
data = "AllDataDf.csv"

df = pd.read_csv(os.path.join(dataFolder, data))

In [127]:
# fix index from read_csv
df = df.rename(columns={'Unnamed: 0': 'Seq'})
df = df.set_index('Seq')

In [128]:
df.head(10)

,OdomPosX,OdomPosY,OdomOrientZ,OdomOrientW,OdomLinX,OdomAngZ,ImuOrientX,ImuOrientY,ImuOrientZ,ImuOrientW,ImuAngVelX,ImuAngVelY,ImuAngVelZ,ImuAccelX,ImuAccelY,ImuAccelZ,TimeDelta2,TimeDelta4,TimeDelta8,TimeDelta16,TimeDelta32,TimeDelta64,OdomPosXDelta2,OdomPosXDelta4,OdomPosXDelta8,OdomPosXDelta16,OdomPosXDelta32,OdomPosXDelta64,OdomPosYDelta2,OdomPosYDelta4,OdomPosYDelta8,OdomPosYDelta16,OdomPosYDelta32,OdomPosYDelta64,OdomOrientZDelta2,OdomOrientZDelta4,OdomOrientZDelta8,OdomOrientZDelta16,OdomOrientZDelta32,OdomOrientZDelta64,OdomOrientWDelta2,OdomOrientWDelta4,OdomOrientWDelta8,OdomOrientWDelta16,OdomOrientWDelta32,OdomOrientWDelta64,OdomLinXDelta2,OdomLinXDelta4,OdomLinXDelta8,OdomLinXDelta16,OdomLinXDelta32,OdomLinXDelta64,OdomAngZDelta2,OdomAngZDelta4,OdomAngZDelta8,OdomAngZDelta16,OdomAngZDelta32,OdomAngZDelta64,ImuOrientXDelta2,ImuOrientXDelta4,...,ImuOrientXDelta16,ImuOrientXDelta32,ImuOrientXDelta64,ImuOrientYDelta2,ImuOrientYDelta4,ImuOrientYDelta8,ImuOrientYDelta16,ImuOrientYDelta32,ImuOrientYDelta64,ImuOrientZDelta2,ImuOrientZDelta4,ImuOrientZDelta8,ImuOrientZDelta16,ImuOrientZDelta32,ImuOrientZDelta64,ImuOrientWDelta2,ImuOrientWDelta4,ImuOrientWDelta8,ImuOrientWDelta16,ImuOrientWDelta32,ImuOrientWDelta64,ImuAngVelXDelta2,ImuAngVelXDelta4,ImuAngVelXDelta8,ImuAngVelXDelta16,ImuAngVelXDelta32,ImuAngVelXDelta64,ImuAngVelYDelta2,ImuAngVelYDelta4,ImuAngVelYDelta8,ImuAngVelYDelta16,ImuAngVelYDelta32,ImuAngVelYDelta64,ImuAngVelZDelta2,ImuAngVelZDelta4,ImuAngVelZDelta8,ImuAngVelZDelta16,ImuAngVelZDelta32,ImuAngVelZDelta64,ImuAccelXDelta2,ImuAccelXDelta4,ImuAccelXDelta8,ImuAccelXDelta16,ImuAccelXDelta32,ImuAccelXDelta64,ImuAccelYDelta2,ImuAccelYDelta4,ImuAccelYDelta8,ImuAccelYDelta16,ImuAccelYDelta32,ImuAccelYDelta64,ImuAccelZDelta2,ImuAccelZDelta4,ImuAccelZDelta8,ImuAccelZDelta16,ImuAccelZDelta32,ImuAccelZDelta64,Speed,Terrain,Trial
Seq,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,101.055844,7.616985,0.274688,0.961533,0.151574,-0.000236,0.000515,-0.013383,-0.542873,0.839703,0.019156,0.040440,-0.005321,0.225055,0.058059,10.169971,0.008933,0.027088,0.045897,0.100897,0.193743,0.392171,0.001234,0.002468,0.004935,0.012789,0.026364,0.050366,0.000768,0.001536,0.003071,0.007954,0.016421,0.031518,-0.000008,-0.000010,-0.000001,0.000127,-0.001574,-0.003692,0.000002,0.000003,3.405980e-07,-0.000036,0.000451,0.001062,0.001296,0.001820,0.001325,0.000271,-0.004892,-0.001824,-0.000807,-0.002108,-0.005697,0.016504,-0.006388,-0.000171,-0.000342,-0.000598,...,0.002573,-0.001060,0.001021,0.000349,-0.000047,0.000171,0.005194,0.001072,0.002645,-0.000008,-2.980232e-05,-0.000021,-0.000001,-0.001408,-0.003269,5.364418e-07,-1.943111e-05,-0.000010,0.000101,-0.000889,-0.002058,-0.001064,-0.001064,0.002128,0.019156,0.011174,0.017027,-0.026605,-0.001064,0.089393,0.114934,0.046293,0.032990,0.005321,0.003193,-1.809140e-02,-0.010642,0.000532,-0.005321,-0.077213,-0.089184,0.347159,0.433949,0.384569,-0.465373,-0.108338,-0.156820,0.049680,-0.055665,-0.148141,-0.256778,-0.481833,-1.060631,0.010176,0.308852,-0.015562,0.110433,15,Turf,2
1,101.056461,7.617369,0.274684,0.961535,0.152222,-0.000639,0.000125,-0.012841,-0.542871,0.839713,0.014899,0.000000,0.000000,0.180164,0.053870,9.819818,0.008593,0.020947,0.045709,0.100557,0.192423,0.388461,0.001234,0.002468,0.004935,0.012776,0.026166,0.050366,0.000768,0.001536,0.003071,0.007947,0.016295,0.031511,-0.000008,-0.000017,-0.000008,0.000125,-0.001441,-0.003685,0.000002,0.000005,2.179827e-06,-0.000036,0.000413,0.001060,0.001296,0.002591,0.002097,0.000483,-0.002097,-0.000744,-0.000807,-0.001614,-0.005203,0.010766,0.003111,0.000100,-0.000390,-0.000860,...,0.001806,-0.001685,0.000676,0.000542,0.000692,0.000712,0.005405,0.001342,0.003567,0.000001,-1.785159e-05,-0.000019,0.000013,-0.001384,-0.003271,9.298325e-06,-1.490116e-07,-0.000001,0.000110,-0.000870,-0.002044,-0.004257,-0.005321,-0.002128,0.020220,0.005321,0.014899,-0.040440,-0.054274,0.048953,

In [129]:
# check for none values and correct size
print(df.isnull().sum().sum()) # check for NaN
df.shape

0


(252489, 121)

### Feature engineering: elimination

In [130]:
# eliminate columns derived from time, and from angular velocity
columnsToDrop = [x for x in df.columns.tolist() if 'Time' in x or ('Delta' not in x and 'Odom' in x) ]
df = df.drop(columns=columnsToDrop)

### Speed filtering

In [131]:
df = df[df['Speed']==10]

### Logistic Regression
#### Now that we have processed the data, we can apply logistic regression to classify.

In [132]:
# sort the train test split where train on t1 test on t2
dfTrain = df[df['Trial']==2]
dfTest = df[df['Trial']==1]

Y_train = dfTrain['Terrain']
Y_test = dfTest['Terrain']

X_train = dfTrain.drop(columns=['Terrain', 'Speed', 'Trial'])
X_test = dfTest.drop(columns=['Terrain', 'Speed', 'Trial'])

In [133]:
# randomly split to train and test
# # separate labels
# terrains = df['Terrain']
# speeds = df['Speed']
# df= df.drop(columns=['Terrain', 'Speed'])

# X = df
# Y = terrains
# X_test, X_train, Y_test, Y_train = train_test_split(X, Y, test_size=0.5)

In [134]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(30302, 106)
(30302,)
(30154, 106)
(30154,)


In [135]:
model = LogisticRegression()
model.fit(X_train, Y_train)

C:\Users\jaker\Anaconda3\envs\tf-p37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

C:\Users\jaker\Anaconda3\envs\tf-p37\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning:

Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [136]:
print("Accuracy on test set is: {}".format(model.score(X_test, Y_test)))

Accuracy on test set is: 0.4962194070438416
